# コンコーダンサー(Simple)

このスクリプトは、spaCyライブラリを使用して事前に解析されたテキストデータに対してコンコーダンス検索を行い、検索結果をHTMLファイルやテキストファイルに出力する。また、検索結果をソートする機能も備える。

## 使用方法

### 解析データの読み込み

1. 解析済みのテキストデータ（.spacyファイル）とそれに対応するテキストファイル（.txt）を用意する。
2. スクリプト内の「ユーザーが設定するパラメーター」セクションで以下の変数を設定する。
   - `main_directory`: 親ディレクトリへの相対パス
   - `subdirectories`: 読み込む子ディレクトリ名のリスト
   - `nlp_model`: 解析に利用したモデル名
   - `selected_file_names`: 特定のファイルのみを読み込む場合はここにtxtファイル名を追加（空リストの場合は全ファイルを読み込む）


### コンコーダンサーの実行

1. スクリプト内の「ユーザーが設定するパラメーター」セクションで以下の変数を設定する。
   - `search_type`: 検索タイプ（'word', 'lemma', 'phrase', 'regex'）
   - `input_keyword`: 検索するキーワード
   - `part_of_speech`: 品詞（word, lemma検索で使用可能, 'NOUN', 'VERB' 等、または None）
   - `window_size_left`, `window_size_right`: KWIC表示の左右のウィンドウサイズ
   - `language_direction`: 言語の方向（"ltr" または "rtl"）
   - `output_html_file`: 出力するHTMLファイル名
2. スクリプトを実行すると、指定した設定に基づいて検索が行われ、結果が出力されます。


## 追加機能

このスクリプトには、以下のような追加機能が含まれる。

- ソート機能: 検索結果をキーワードや周辺の単語を基準にソートできる。
- テキストファイル出力機能: 検索結果をテキストファイルに出力できる。Window単位での出力と文単位での出力が可能。
- CSV形式での出力: 検索結果をCSV形式で出力できる。

# **ライブラリのインポート(最初に実行)**

In [1]:
import os
import spacy
from spacy.tokens import DocBin
from spacy.matcher import PhraseMatcher
import re
import csv

# **解析データの読み込み**
## ユーザーが設定するパラメーター

In [2]:
# 基本設定
main_directory = "processed_data"  # 親ディレクトリへの相対パス
subdirectories = ["sub1"]  # 読み込む子ディレクトリ名のリスト
nlp_model = "en_core_web_sm"  # 解析に利用したモデル名

# 特定のファイルのみを読み込む場合は以下のリストにtxtファイル名を追加(空リストの場合は全ファイルを読み込む)
selected_file_names = []

## 解析データ読み込みの実行

In [ ]:
# spaCyの言語モデルをロード
nlp = spacy.load(nlp_model)

# 解析結果を格納する辞書の初期化
docs_dict = {}
# フィルタリングされたDocオブジェクトを格納する辞書の初期化
filtered_docs = {}

# 指定されたサブディレクトリ内のファイルを読み込む
for subdir in subdirectories:
    directory = os.path.join(main_directory, subdir)
    
    # サブディレクトリ内のファイルを読み込む
    for filename in os.listdir(directory): 
        if filename.endswith(".spacy"):  # .spacyファイルならば
            spacy_path = os.path.join(directory, filename)
            txt_filename = filename.replace(".spacy", ".txt")  # .spaCyファイルに対応するテキストファイル名を作成
            txt_path = os.path.join(directory, txt_filename)  # .spacyファイルに対応するテキストファイルのパス
            
            # .spacyファイルからDocオブジェクトを読み込む
            doc_bin = DocBin().from_disk(spacy_path)  # DocBinオブジェクトを読み込む
            docs = list(doc_bin.get_docs(nlp.vocab))  # Docオブジェクトをリストに格納
            
            # 対応する.txtファイルからファイル名を読み込む
            if os.path.exists(txt_path):
                with open(txt_path, 'r', encoding='utf-8') as f:
                    file_names = [line.strip() for line in f.readlines()]  # ファイル名をリストに格納
                
                # Docオブジェクトとファイル名の対応を辞書に格納
                for doc, fname in zip(docs, file_names):
                    docs_dict[fname] = doc
            else:
                print(f"警告: {txt_filename} に対応するテキストファイルが存在しません。")

# ユーザーがファイル名を指定した場合、それに対応するDocオブジェクトをフィルタリング
if selected_file_names:
    missing_files = []  # 存在しないファイル名を格納するリスト
    for fname in selected_file_names:
        if fname in docs_dict:  # 指定されたファイル名が読み込んだデータに存在する場合
            filtered_docs[fname] = docs_dict[fname]
        else:
            missing_files.append(fname)
    
    if missing_files:
        print(f"警告: 次の指定されたファイルは存在しません: {', '.join(missing_files)}")
    if filtered_docs:
        print("指定された一部のデータが読み込まれました。")
    else:
        print("指定されたファイルに対応するデータが見つかりませんでした。")
else:
    filtered_docs = docs_dict
    print("全てのデータが読み込まれました。")
print(f"読み込まれたデータ数: {len(filtered_docs)}")
print(f"読み込まれたファイル名: {list(filtered_docs.keys())}")

# **コンコーダンサー**
## ユーザーが設定するパラメータ―

In [16]:
# 基本設定
search_type = 'phrase'    # 検索タイプ（'word', 'lemma', 'phrase', 'regex'）
input_keyword = "hello world"  # 検索するキーワード

# 追加設定
part_of_speech = None        # 品詞（word, lemma検索で使用可能, 'NOUN', 'VERB' 等、または None）

# window sizeを指定(左右に何単語分表示するか)
window_size_left = 15       # KWIC表示の左側のウィンドウサイズ
window_size_right = 15      # KWIC表示の右側のウィンドウサイズ

# 言語の設定
language_direction = "ltr"  # "ltr" (left-to-right) または "rtl" (right-to-left)

# 出力するHTMLファイル名
output_html_file = "kwic_results.html"

## 検索の実行

In [ ]:
def search_word(doc, keyword, part_of_speech, window_size_left, window_size_right, keyword_positions, file_name):
    """
    単語レベルでキーワードを検索し、KWIC結果を返す関数。

    Args:
        doc (Doc): spaCyのDocオブジェクト。
        keyword (str): 検索するキーワード。
        part_of_speech (str or None): 検索する品詞。Noneの場合は品詞を指定しない。
        window_size_left (int): 左側のコンテキストのウィンドウサイズ。
        window_size_right (int): 右側のコンテキストのウィンドウサイズ。
        keyword_positions (list): キーワードの位置情報とファイル名を格納するリスト。
        file_name (str): 検索対象のファイル名。

    Returns:
        list: KWIC結果のリスト。[(left_context, keyword, right_context), ...]
    """
    results = []  # 結果を格納するリスト
    for token in doc:
        if part_of_speech is None or token.pos_ == part_of_speech:  # 品詞が指定されていないか、指定された品詞の場合
            if token.text.lower() == keyword.lower():  # キーワードが一致する場合(小文字で比較)
                # 左右のコンテキストをトークンのリストとして取得
                left_context = [t.text_with_ws for t in doc[max(0, token.i - window_size_left):token.i]]  # 左側のコンテキスト(maxを使用して負のインデックスを防ぐ)
                right_context = [t.text_with_ws for t in doc[token.i + 1:token.i + 1 + window_size_right]] # 右側のコンテキスト

                # 結果をリストに追加
                results.append((left_context, [token.text], right_context, file_name))
                keyword_positions.append((token.i, file_name))  # キーワードの位置情報とファイル名を追加
    return results

def search_lemma(doc, keyword, part_of_speech, window_size_left, window_size_right, keyword_positions, file_name):
    """
    レンマレベルでキーワードを検索し、KWIC結果を返す関数。

    Args:
        doc (Doc): spaCyのDocオブジェクト。
        keyword (str): 検索するキーワード。
        part_of_speech (str or None): 検索する品詞。Noneの場合は品詞を指定しない。
        window_size_left (int): 左側のコンテキストのウィンドウサイズ。
        window_size_right (int): 右側のコンテキストのウィンドウサイズ。
        keyword_positions (list): キーワードの位置情報とファイル名を格納するリスト。
        file_name (str): 検索対象のファイル名。

    Returns:
        list: KWIC結果のリスト。[(left_context, keyword, right_context), ...]
    """
    results = []  # 結果を格納するリスト
    for token in doc:
        if part_of_speech is None or token.pos_ == part_of_speech:
            if token.lemma_.lower() == keyword.lower(): # キーワードが一致する場合(小文字で比較, lemma_属性を使用)
                left_context = [t.text_with_ws for t in doc[max(0, token.i - window_size_left):token.i]]
                right_context = [t.text_with_ws for t in doc[token.i + 1:token.i + 1 + window_size_right]]

                # 結果をリストに追加
                results.append((left_context, [token.text], right_context, file_name))
                keyword_positions.append((token.i, file_name))
    return results

def search_phrase(doc, phrase, window_size_left, window_size_right, nlp, keyword_positions, file_name):
    """
    フレーズレベルでキーワードを検索し、KWIC結果を返す関数。

    Args:
        doc (Doc): spaCyのDocオブジェクト。
        phrase (str): 検索するフレーズ。
        window_size_left (int): 左側のコンテキストのウィンドウサイズ。
        window_size_right (int): 右側のコンテキストのウィンドウサイズ。
        nlp (Language): spaCyの言語モデルオブジェクト。
        keyword_positions (list): キーワードの位置情報とファイル名を格納するリスト。
        file_name (str): 検索対象のファイル名。

    Returns:
        list: KWIC結果のリスト。[(left_context, keyword, right_context), ...]
    """
    matcher = PhraseMatcher(nlp.vocab, attr='LOWER')  # 小文字でのマッチング
    phrase_doc = nlp.make_doc(phrase.lower())  # nlp.make_docを使用してトークナイズのみ実行
    matcher.add("PHRASE", [phrase_doc])  # マッチャーにフレーズを追加
    matches = matcher(doc)  # マッチを実行

    results = []
    for match_id, start, end in matches:
        # コンテキストをトークンのリストとして取得し、text_with_ws属性を使用して空白情報を含める
        left_context_tokens = [token.text_with_ws for token in doc[max(0, start - window_size_left):start]]
        right_context_tokens = [token.text_with_ws for token in doc[end:end + window_size_right]]
        matched_phrase = [token.text_with_ws for token in doc[start:end]]  # マッチしたフレーズを取得

        # 結果をリストに追加
        results.append((left_context_tokens, matched_phrase, right_context_tokens, file_name))
        keyword_positions.append((start, file_name))
    
    return results

def search_regex(doc, keyword, window_size_left, window_size_right, keyword_positions, file_name):
    """
    正規表現を用いてキーワードを検索し、KWIC結果を返す関数。

    Args:
        doc (Doc): spaCyのDocオブジェクト。
        keyword (str): 検索する正規表現パターン。
        window_size_left (int): 左側のコンテキストのウィンドウサイズ。
        window_size_right (int): 右側のコンテキストのウィンドウサイズ。
        keyword_positions (list): キーワードの位置情報とファイル名を格納するリスト。
        file_name (str): 検索対象のファイル名。

    Returns:
        list: KWIC結果のリスト。[(left_context, keyword, right_context), ...]
    """
    results = []
    
    for match in re.finditer(keyword, doc.text): # 正規表現パターンにマッチするキーワードを検索
        start_idx = match.start() # マッチするキーワードの開始インデックス
        end_idx = match.end()   # マッチするキーワードの終了インデックス
        
        # 左右のコンテキストを取得
        start_token_idx = [token.i for token in doc if token.idx <= start_idx][-1] # start_idxに最も近いトークンのインデックスを取得(キーワードの最初のトークンのインデックス)
        end_token_idx = [token.i for token in doc if token.idx < end_idx][-1] # end_idxに最も近いトークンのインデックスを取得(キーワードの最後のトークンのインデックス)

        # 左右のコンテキストをトークンのリストとして取得し、text_with_wsを使用
        left_context = [t.text_with_ws for t in doc[max(0, start_token_idx - window_size_left):start_token_idx]]
        right_context = [t.text_with_ws for t in doc[end_token_idx + 1:end_token_idx + 1 + window_size_right]]
        keyword_token = [t.text_with_ws for t in doc[start_token_idx:end_token_idx + 1]] # キーワードのトークンを取得

        
        results.append((left_context, keyword_token, right_context, file_name))
        keyword_positions.append((start_token_idx, file_name))
    
    return results

def perform_kwic_search(doc, keyword, search_type, file_name, keyword_positions, nlp=None, window_size_left=5, window_size_right=5, part_of_speech=None):
    """
    指定された検索タイプに基づいてKWIC検索を実行し、結果を返す関数。

    Args:
        doc (Doc): spaCyのDocオブジェクト。
        keyword (str): 検索するキーワード。
        search_type (str): 検索タイプ。'word', 'lemma', 'phrase', 'regex'のいずれか。
        file_name (str): 検索対象のファイル名。
        keyword_positions (list): キーワードの位置情報とファイル名を格納するリスト。
        nlp (Language, optional): spaCyの言語モデルオブジェクト。デフォルトはNone。
        window_size_left (int, optional): 左側のコンテキストのウィンドウサイズ。デフォルトは5。
        window_size_right (int, optional): 右側のコンテキストのウィンドウサイズ。デフォルトは5。
        part_of_speech (str or None, optional): 検索する品詞。デフォルトはNone。

    Returns:
        tuple: (KWIC結果のリスト, キーワードの位置情報とファイル名のリスト)
    """

    if search_type == 'word':
        results = search_word(doc, keyword, part_of_speech, window_size_left, window_size_right, keyword_positions, file_name)
    
    elif search_type == 'lemma':
        results = search_lemma(doc, keyword, part_of_speech, window_size_left, window_size_right, keyword_positions, file_name)
    
    elif search_type == 'phrase':
        results = search_phrase(doc, keyword, window_size_left, window_size_right, nlp, keyword_positions, file_name)
    
    elif search_type == 'regex':
        results = search_regex(doc, keyword, window_size_left, window_size_right, keyword_positions, file_name)
    
    return results, keyword_positions


all_kwic_results = [] # すべてのKWIC結果を格納するリスト(例: [(left_context, keyword, right_context, file_name), ...])
keyword_positions = [] # キーワードの位置情報を格納するリスト(例: [(position, file_name), ...])

if part_of_speech is not None and (part_of_speech.lower() == "none" or part_of_speech == "" or search_type in ['phrase', 'regex']):
    part_of_speech = None # 品詞が指定されていない場合、Noneに設定

if search_type not in ["word", "lemma", "phrase", "regex"]: # 検索タイプが不正な場合、エラーを発生
    raise ValueError("検索タイプは 'word', 'lemma', 'phrase', 'regex' のいずれかである必要があります。")

for file_name, doc in filtered_docs.items():
    results, keyword_positions = perform_kwic_search(
        doc, input_keyword, search_type, file_name, keyword_positions, nlp, window_size_left, window_size_right, part_of_speech)
    all_kwic_results.extend(results)  # KWIC結果をリストに追加


# 右から左に書く言語では、right_contextとleft_contextを入れ替える
if language_direction == "rtl":
    all_kwic_results = [(right_context, keyword, left_context, file_name) for left_context, keyword, right_context, file_name in all_kwic_results]

def output_kwic_results_to_html(kwic_results, output_html_file, file_info=True):
    """
    KWIC結果をHTMLファイルに出力する関数。

    Args:
        kwic_results (list): KWIC結果のリスト。[(left_context, keyword, right_context, file_name), ...]
        output_html_file (str): 出力するHTMLファイルの名前。
        file_info (bool, optional): ファイル情報を出力するかどうか。デフォルトはTrue。
    """
    with open(output_html_file, "w", encoding="utf-8") as outfile:
        # HTMLのヘッダーとスタイルシートを書き込み
        outfile.write("<!DOCTYPE html>\n<html>\n<head>\n<meta charset='UTF-8'>\n")
        outfile.write('<meta name="viewport" content="width=device-width, initial-scale=0.75">\n')
        outfile.write("<title>KWIC Results</title>\n<style>\n")
        outfile.write("body { font-family: Arial, sans-serif; margin: 20px; }\n")
        outfile.write("table { width: 100%; border-collapse: collapse; }\n")
        outfile.write("th, td { border: 1px solid #999; padding: 0.5rem; }\n")
        outfile.write("td.line-number { text-align: right; font-size: smaller; color: #777; }\n")
        outfile.write("td.left { text-align: right; }\n")
        outfile.write("td.keyword { text-align: center; font-weight: bold; background-color: #e8e8e8; }\n")
        outfile.write("td.right { text-align: left; }\n")
        outfile.write("</style>\n</head>\n<body>\n")
        outfile.write("<h1>KWIC Analysis Results</h1>\n")
        outfile.write("<table>\n<thead>\n<tr>\n<th>#</th>\n<th>Left Context</th>\n<th>Keyword</th>\n<th>Right Context</th>\n</tr>\n</thead>\n<tbody>\n")

        # 結果をテーブルの行として出力
        for i, (left_context, kw, right_context, source_file) in enumerate(kwic_results, 1):
            left_str = ''.join(left_context).rstrip()
            right_str = ''.join(right_context).rstrip()
            keyword_str = ''.join(kw).rstrip()
            outfile.write(f"<tr>\n<td class='line-number'>{i}</td>\n<td class='left'>{left_str}</td>\n<td class='keyword'>{keyword_str}</td>\n<td class='right'>{right_str}</td>\n</tr>\n")

        outfile.write("</tbody>\n</table>\n")

        # ファイル情報のセクションを追加（オプション）
        if file_info:
            outfile.write("<h2>File Information</h2>\n")
            outfile.write("<ul>\n")
            for i, (_, _, _, filename) in enumerate(kwic_results, 1):
                outfile.write(f"<li>Line {i}: {filename}</li>\n")
            outfile.write("</ul>\n")

        outfile.write("</body>\n</html>")

if all_kwic_results:
    # コンソールに検索結果の概要を表示
    print(f"検索結果: {len(all_kwic_results)}件 (検索モード: {search_type}, キーワード: {input_keyword}, 品詞: {part_of_speech if part_of_speech else 'なし'})")
    
    # 検索結果をHTMLファイルに出力する関数を呼び出し
    output_kwic_results_to_html(all_kwic_results, output_html_file)
    print(f"検索結果を '{output_html_file}' に出力しました。")
else: 
    # 検索結果がなかった場合のメッセージをコンソールに表示
    print(f"'{input_keyword}'に一致する結果は見つかりませんでした。(検索モード: {search_type}, 品詞: {part_of_speech if part_of_speech else 'なし'})")

# **ソート機能**
## ユーザーが設定するパラメーター
ソートレベルを指定する（例:["K", "1R", "1L", "2R"]）   
リストの最初の要素が主要なソート基準、次の要素が次のソート基準となる。  
"1R"はキーワードの右側の最初の単語、"2R"は右側の2番目の単語を基準にソート。  
"1L"はキーワードの左側の最後の単語、"2L"は左側の最後から2番目の単語を基準にソート。  
"K"はキーワード自体を基準にソート。  

In [6]:
sort_levels = ["1K", "1R", "2R", "3R"]  # ソートのレベルを指定
output_sorted_html_file = "sorted_kwic_results.html" # ソートされた結果を出力するHTMLファイル名

## ソートの実行

In [ ]:
def sort_kwic_results(kwic_results, sort_levels):
    """
    KWIC分析結果をソートする。

    Args:
        kwic_results (list): (left_context, keyword, right_context, file_name)のタプルを含むリスト。
        sort_levels (list): ソートのレベルを指定するリスト。

    Returns:
        list: ソートされたKWIC分析結果。
    """
    def get_sort_key(item):
        left, kw, right, _ = item
        keys = []
        for level in sort_levels:
            index, direction = int(level[:-1]) - 1, level[-1]  # '1K', '1R', '1L' の形式を解析
            
            if direction == 'K':
                # キーワードリストからソートキーを取得
                keys.append(kw[index] if index < len(kw) else "")
            elif direction == 'R':
                # 右コンテキストリストからソートキーを取得
                keys.append(right[index] if index < len(right) else "")
            elif direction == 'L':
                # 左コンテキストリストから逆順でソートキーを取得
                keys.append(left[-index-1] if index < len(left) else "")
            
        return tuple(keys)
    
    return sorted(kwic_results, key=get_sort_key)

# ソートされた結果をHTMLファイルに出力
sorted_kwic_results = sort_kwic_results(all_kwic_results, sort_levels)
# 上記セルで定義した関数を使用してHTMLファイルに出力
output_kwic_results_to_html(sorted_kwic_results, output_sorted_html_file)
print(f"ソートされた結果が {output_sorted_html_file} に出力されました。")

# **テキストファイル出力機能**

## コンコーダンス検索結果の出力（window＿size）

In [ ]:
# 出力するテキストファイルの名前を定義
output_text_file = "kwic_results.txt"

# テキストファイルに結果を出力
with open(output_text_file, "w", encoding="utf-8") as outfile:
    for left_context, kw, right_context, _ in all_kwic_results:
        # コンテキストとキーワードを結合
        left_str = ''.join(left_context).rstrip()
        right_str = ''.join(right_context).rstrip()
        keyword_str = ''.join(kw).rstrip()
        outfile.write(f"{left_str} [{keyword_str}] {right_str}\n") # 出力形式は他の形式にも変更可能

print(f"検索結果を {output_text_file} に出力しました。")

## ソート結果の出力（window＿size）

In [ ]:
# 出力するテキストファイルの名前を定義
sorted_output_text_file = "sorted_kwic_results.txt"

# ソートされたKWIC分析結果をテキストファイルに出力
with open(sorted_output_text_file, "w", encoding="utf-8") as outfile:
    for left_context, kw, right_context, _ in sorted_kwic_results:
        # コンテキストとキーワードを結合
        left_str = ''.join(left_context).rstrip()
        right_str = ''.join(right_context).rstrip()
        keyword_str = ''.join(kw).rstrip()
        # パディングなしで出力
        outfile.write(f"{left_str} [{keyword_str}] {right_str}\n")

print(f"ソートされた検索結果を {sorted_output_text_file} に出力しました。")

## 文単位での出力

In [ ]:
# 出力するテキストファイルの名前を定義
sentences_text_file = "extracted_sentences.txt"

output_sentences = set()  # 出力済みの文を追跡するためのセット
duplicated_sentences = []  # 重複する文を追跡するためのリスト

# テキストファイルに出力するためのファイルを開く
with open(sentences_text_file, "w", encoding="utf-8") as f:
    for position, file_name in keyword_positions:
        # 対応するドキュメントを取得
        if file_name in filtered_docs:
            doc = filtered_docs[file_name]  # ファイル名に対応するDocオブジェクトを取得
            for sent in doc.sents:
                if position >= sent.start and position < sent.end: # キーワードが含まれる文を見つけたら
                    sent_text = str(sent) # 文を文字列に変換
                    if sent_text in output_sentences:  # 文が既に出力されている場合、重複リストに追加
                        duplicated_sentences.append(sent_text)
                    else:
                        f.write(sent_text)  # まだ出力されていない文ならファイルに書き込む
                        f.write('\n')  # 改行を挿入
                        output_sentences.add(sent_text)  # 出力済みの文を追跡するためのセットに追加
                    break  # 一致した文が見つかったらループを抜ける
        else:
            print(f"警告: {file_name} に対応するドキュメントが見つかりませんでした。")

print(f"抽出された文が {sentences_text_file} に出力されました。")

# 重複する文のリストを表示（重複文の数、内容）
print(f"重複する文の数: {len(duplicated_sentences)}")
for sent in duplicated_sentences:
    print(sent)

## CSV形式での出力(通常の検索結果)

In [ ]:
# 出力するCSVファイルの名前を定義
output_csv_file = "kwic_results.csv"

# CSVファイルに結果を出力するための関数
def output_kwic_to_csv(kwic_results, output_csv_file):
    with open(output_csv_file, 'w', newline='', encoding='utf-8') as csvfile:
        csvwriter = csv.writer(csvfile, delimiter=',', quotechar='"', quoting=csv.QUOTE_MINIMAL)
        
        # CSVのヘッダーを書き込む
        csvwriter.writerow(['Left Context', 'Keyword', 'Right Context'])
        
        # 各KWIC結果をCSVファイルに書き込む
        for left_context, keyword, right_context, _ in kwic_results:
            left_str = ''.join(left_context).rstrip()
            right_str = ''.join(right_context).rstrip()
            keyword_str = ''.join(keyword).rstrip()
            csvwriter.writerow([left_str, keyword_str, right_str])

# ソートされていない結果を出力
output_kwic_to_csv(all_kwic_results, output_csv_file)
print(f"検索結果が {output_csv_file} に出力されました。")

## CSV形式での出力(ソートした検索結果)

In [ ]:
sorted_output_csv_file = "sorted_kwic_results.csv"

# CSVファイルに結果を出力するための関数
def output_kwic_to_csv(kwic_results, output_csv_file):
    with open(output_csv_file, 'w', newline='', encoding='utf-8') as csvfile:
        csvwriter = csv.writer(csvfile, delimiter=',', quotechar='"', quoting=csv.QUOTE_MINIMAL)
        
        # CSVのヘッダーを書き込む
        csvwriter.writerow(['Left Context', 'Keyword', 'Right Context'])
        
        # 各KWIC結果をCSVファイルに書き込む
        for left_context, keyword, right_context, _ in kwic_results:
            left_str = ''.join(left_context).rstrip()
            right_str = ''.join(right_context).rstrip()
            keyword_str = ''.join(keyword).rstrip()
            csvwriter.writerow([left_str, keyword_str, right_str])

# ソートされた結果を出力
output_kwic_to_csv(sorted_kwic_results, sorted_output_csv_file)
print(f"ソートされた検索結果が {sorted_output_csv_file} に出力されました。")